<a href="https://colab.research.google.com/github/Ramjas-Langdi/Topsis_text_conversation_102117159/blob/main/text_conversation_102117159.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers pandas


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch

models = [
    {"name": "bert-base-uncased", "model": "textattack/bert-base-uncased-MNLI"},
    {"name": "bert-base-cased", "model": "bert-base-cased"},
    {"name": "roberta-base", "model": "roberta-base"},
    {"name": "distilbert-base-uncased", "model": "distilbert-base-uncased"},
    {"name": "xlnet-base-cased", "model": "xlnet-base-cased"}
]

parameters = ["coherence", "engagement", "relevance", "understandability"]

text_conversation = [
    "User: Hello! How can I help you today?",
    "Bot: Hi there! I'm having some trouble with my account.",
    "User: Sure, I'll do my best to assist you. What seems to be the problem?",
    "Bot: I keep getting an error message when I try to log in.",
    "User: Let me check your account details to see what might be causing the issue.",
    "Bot: Thank you for your help."
]

scores = np.zeros((len(models), len(parameters)))

for i, model_info in enumerate(models):
    tokenizer = AutoTokenizer.from_pretrained(model_info["model"])
    model = AutoModel.from_pretrained(model_info["model"])

    embeddings = []
    for utterance in text_conversation:
        inputs = tokenizer(utterance, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            output = model(**inputs)
            embeddings.append(output.last_hidden_state.mean(dim=1).squeeze().numpy())

    conversation_embedding = np.mean(embeddings, axis=0)

    ideal_solution = np.max(conversation_embedding, axis=0)
    anti_ideal_solution = np.min(conversation_embedding, axis=0)

    distance_to_ideal = np.linalg.norm(conversation_embedding - ideal_solution)
    distance_to_anti_ideal = np.linalg.norm(conversation_embedding - anti_ideal_solution)

    performance_score = distance_to_anti_ideal / (distance_to_ideal + distance_to_anti_ideal)

    scores[i] = performance_score

ranks = np.argsort(scores.sum(axis=1))
df = pd.DataFrame(scores, columns=parameters)
df.insert(0, "Model", [model["name"] for model in models])
df["TOPSIS Score"] = scores.sum(axis=1)
df["Rank"] = len(models) - ranks

df.to_csv("102117159model_comparison_results.csv", index=False)

print(df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the result file
df = pd.read_csv("model_comparison_results.csv")

# Set the model names as index
df.set_index("Model", inplace=True)

# Plotting
df.plot(kind="bar", figsize=(12, 6))
plt.title("Comparison of Models based on Evaluation Parameters")
plt.xlabel("Models")
plt.ylabel("Scores")
plt.xticks(rotation=45, ha="right")
plt.legend(title="Parameters")
plt.tight_layout()
plt.show()